In [50]:
# import os
import SunStatistics as SS
import Old as OS
# import scipy.stats as stats
import numpy as np
import pandas as pd
# from matplotlib import pyplot as plt
from datetime import timedelta,datetime
# from math import floor
# import seaborn as sns
# from matplotlib.colors import LinearSegmentedColormap

# import datetime as dt
import pytz
# from sklearn.utils import resample
# %matplotlib notebook
# import warnings
# from sklearn.model_selection import train_test_split
# from sklearn.neural_network import MLPRegressor as MPR
# from sklearn.preprocessing import Normalizer,StandardScaler,MinMaxScaler
# from sklearn.pipeline import Pipeline
# from sklearn import metrics
# from sklearn.metrics import classification_report,confusion_matrix
# from scipy.optimize import curve_fit
# from matplotlib.ticker import NullFormatter
# import Data_Processor as DP
# import importlib
# importlib.reload(DP)

# from sklearn.model_selection import KFold
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# from IPython.display import display, clear_output
# from ipywidgets import FloatProgress, HTML
G_Drive = 'G:\\My Drive\\'
Output_Path = G_Drive+'FishIsland_Outputs/'

In [65]:
Taglu_Data = Output_Path+'Taglu_Data/Hourly.csv'
AWS = pd.read_csv(Taglu_Data,
parse_dates={'datetime':['Year','Month','Day','Time']}, 
).set_index('datetime')

# Gap Fill
C=['Air Temp','Net Radiation','Net SW Radiation','Net LW Radiation ','Wind Speed','Wind direction']
s = AWS.index.to_series()
print('Missing')
for c in C:
    print(c,AWS.loc[((AWS.index.month>=4)&(AWS.index.month<=10)),c].isnull().sum())
    miss = AWS[c].isna()
    #create consecutive groups
    g = miss.ne(miss.shift()).cumsum()
    #aggregate minimal 
    m1 = s.groupby(g).min()
    #get minimal of next groups, last value is replaced last value of index
    m2 = m1.shift(-1).fillna(AWS.index[-1])
    #get difference, convert to minutes
    out = m2.sub(m1).dt.total_seconds().div(3600).astype(int)
    AWS['Temp_Gap'] = g.map(out)
    AWS['Temp']=AWS[c].interpolate()
    AWS.loc[AWS['Temp_Gap']<=4,c]=AWS.loc[AWS['Temp_Gap']<=4,'Temp']
    AWS['Temp1']=AWS[c].shift(24)
    AWS['Temp2']=AWS[c].shift(-24)
    AWS.loc[AWS['Temp_Gap']>=4,c].fillna(AWS.loc[AWS['Temp_Gap']>=4,['Temp1','Temp2']].mean(axis=1))
AWS = AWS.drop(columns=['Temp_Gap','Temp','Temp1','Temp2'])
AWS_Rename = {'Air Temp':'Air_Temp',
              'Rainfall':'Rainfall',
              'Wind Speed':'Wind_Speed',
              'Wind direction':'Wind_Direction',
              'Net SW Radiation':'Net_SW',
              'Net LW Radiation ':'Net_LW',
              'Net Radiation':'Net_RN'}
AWS = AWS.rename(columns=AWS_Rename)
AWS_cols = list(AWS_Rename.values())
AWS.head()
AWS.index


Missing
Air Temp 40
Net Radiation 40
Net SW Radiation 40
Net LW Radiation  40
Wind Speed 40
Wind direction 298


DatetimeIndex(['2008-08-28 00:00:00', '2008-08-28 01:00:00',
               '2008-08-28 02:00:00', '2008-08-28 03:00:00',
               '2008-08-28 04:00:00', '2008-08-28 05:00:00',
               '2008-08-28 06:00:00', '2008-08-28 07:00:00',
               '2008-08-28 08:00:00', '2008-08-28 09:00:00',
               ...
               '2020-08-11 05:00:00', '2020-08-11 06:00:00',
               '2020-08-11 07:00:00', '2020-08-11 08:00:00',
               '2020-08-11 09:00:00', '2020-08-11 10:00:00',
               '2020-08-11 11:00:00', '2020-08-11 12:00:00',
               '2020-08-11 13:00:00', '2020-08-11 14:00:00'],
              dtype='datetime64[ns]', name='datetime', length=104799, freq=None)

In [66]:
LAT = 69.371182
LON = -134.880935
TZ = -6

Temp = AWS[['Net_SW']].resample('5T').asfreq()
D = Temp.index.floor('D').to_julian_date()
T = Temp.index.hour/24
A = np.ones(D.shape[0])

Zenith,Angle,Angle_Corr,Azimuth,Sunrise,Sunset=SS.SunStats(LAT*A,LON*A,D.values,T.values,TZ*A)
Temp['Sun_Angle'] = Angle_Corr
Temp = Temp.resample('30T').mean()
try:
    AWS = AWS.drop(columns=['Sun_Angle'])
except:
    pass
AWS = AWS.join(Temp[['Sun_Angle']])

AWS['Daytime']=0
AWS.loc[AWS['Sun_Angle']>-0.5,'Daytime']=1

AWS.loc[((AWS['Daytime']==0)&(AWS['Net_SW']>3)),'Daytime']=0.5


Mt = pytz.timezone('Canada/Mountain')
UTC = AWS.index+timedelta(hours=6)
AWS = AWS.set_index(UTC)
AWS.index = AWS.index.tz_localize(pytz.utc).tz_convert(Mt)

AWS.groupby('Daytime').count()['Air_Temp']

C:\Users\wesle\Thesis_Code\SunStatistics.py:25: RuntimeWarning: invalid value encountered in arccos
  W2 = np.degrees(np.arccos(np.cos(np.radians(90.833))/(np.cos(np.radians(LAT))*np.cos(np.radians(T2)))-np.tan(np.radians(LAT))*np.tan(np.radians(T2)))) #HA Sunrise (deg)


Daytime
0.0    49491
0.5      346
1.0    54924
Name: Air_Temp, dtype: int64

In [78]:
EC_Data = pd.read_csv(Output_Path+'FI_Footprints_2022-02-23_2000mx2m.csv',delimiter = ',',header = 0,na_values = -9999,
                   index_col=['datetime'],
                  ).drop('Unnamed: 0',axis=1)
EC_Data = EC_Data.set_index(pd.to_datetime(EC_Data.index, utc=True)).tz_convert(Mt)
EC_Data = EC_Data.rename(columns={'Daytime':'Daytime_Raw',
                                 'Sun_Angle':'Sun_Angle_Raw'})


FSO = pd.read_csv(Output_Path+'Skeeter_et_al_2022_Day_Fix.csv',index_col=['datetime']).drop('Unnamed: 0',axis=1)#.drop('datetime.1',axis=1)

FSO = FSO.set_index(pd.to_datetime(FSO.index, utc=True)).tz_convert(Mt)
FSO = FSO.rename(columns={'NEE_est':'NEE_FSO',
                            'NME_est':'NME_FSO'})
EC_Data = EC_Data.join(FSO[['NEE_FSO',
'NME_FSO']])
EC_Data.head()


,DOY,file_records,used_records,Tau,qc_Tau,rand_err_Tau,H,qc_H,rand_err_H,LE,...,Run,Z_L,canopy_height,d,Collapse,Polygon,Rim,FarField,NEE_FSO,NME_FSO
datetime,,,,,,,,,,,,,,,,,,,,,
2017-06-23 00:00:00-06:00,174,1800.0,1800.0,0.073630,0.0,NaN,6.368283,1.0,NaN,12.286230,...,1,-0.014109,0.117014,0.077229,0.042766,0.577758,0.262825,0.116652,-0.434438,23.892736
2017-06-23 00:30:00-06:00,174,1800.0,1800.0,0.074988,0.0,NaN,-17.662798,0.0,NaN,2.259226,...,1,0.038239,0.117014,0.077229,0.024780,0.582132,0.270226,0.122861,0.261976,19.342712
2017-06-23 01:00:00-06:00,174,1800.0,1800.0,0.108962,0.0,NaN,-11.517344,0.0,NaN,1.372132,...,1,0.014341,0.117014,0.077229,0.020290,0.571270,0.289430,0.119010,0.591992,17.331612
2017-06-23 01:30:00-06:00,174,1800.0,1800.0,0.084511,0.0,NaN,-9.591518,0.0,NaN,NaN,...,1,0.017562,0.117014,0.077229,0.028736,0.565071,0.286668,0.119525,0.598312,18.381430
2017-06-23 02:00:00-06:00,174,1800.0,1800.0,0.070018,0.0,NaN,-8.110602,0.0,NaN,1.213447,...,1,0.019708,0.117014,0.077229,0.031491,0.575233,0.273006,0.120270,0.732068,20.061941


# Merge

In [91]:
EC_Data_r=EC_Data.resample('h').mean()
# EC_Data_r=EC_Data.resample('h',closed='left',label='left').mean()
EC_Data_r = EC_Data_r.drop(['Wind_Direction', 'Wind_Speed'],axis=1)
Data_M1 = EC_Data_r.join(AWS,how='outer')
Data_M1.head()

,DOY,file_records,used_records,Tau,qc_Tau,rand_err_Tau,H,qc_H,rand_err_H,LE,...,JD,Air_Temp,Rainfall,Wind_Speed,Wind_Direction,Net_SW,Net_LW,Net_RN,Sun_Angle,Daytime
datetime,,,,,,,,,,,,,,,,,,,,,
2008-08-28 00:00:00-06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,241,6.0,0.0,3.0,53.88,-0.53,-6.99,-7.51,-5.063956,0.0
2008-08-28 01:00:00-06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,241,5.6,0.0,3.2,56.81,-0.54,-6.18,-6.71,-8.302563,0.0
2008-08-28 02:00:00-06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,241,5.1,0.0,3.2,57.65,-0.55,-6.35,-6.90,-10.355872,0.0
2008-08-28 03:00:00-06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,241,4.7,0.0,2.9,56.24,-0.51,-5.40,-5.90,-11.078917,0.0
2008-08-28 04:00:00-06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,241,4.8,0.0,2.4,68.13,-0.44,-5.17,-5.61,-10.416418,0.0


# ECMWF

In [94]:
ECMWF = pd.read_csv(G_Drive+'earthengine//Climate_Weighted_Mean_long.csv',
                    parse_dates=['system:index'],na_values=-9999,index_col=['system:index'])
ECMWF.index = ECMWF.index.tz_localize(pytz.utc).tz_convert(Mt)
print(ECMWF.columns)
Data_M2 = Data_M1.join(ECMWF.loc[ECMWF.index.year>=2008],how = 'outer')
Data_M2.head()
ECMWF.head()

Index(['date', 'dewpoint_temperature_2m', 'snow_cover',
       'soil_temperature_level_1', 'soil_temperature_level_2',
       'soil_temperature_level_3', 'surface_latent_heat_flux_hourly',
       'surface_net_solar_radiation_hourly',
       'surface_net_thermal_radiation_hourly', 'surface_pressure',
       'surface_sensible_heat_flux_hourly',
       'surface_solar_radiation_downwards_hourly',
       'surface_thermal_radiation_downwards_hourly', 'temperature_2m',
       'total_evaporation_hourly', 'total_precipitation_hourly',
       'u_component_of_wind_10m', 'v_component_of_wind_10m',
       'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2',
       '.geo'],
      dtype='object')


,date,dewpoint_temperature_2m,snow_cover,soil_temperature_level_1,soil_temperature_level_2,soil_temperature_level_3,surface_latent_heat_flux_hourly,surface_net_solar_radiation_hourly,surface_net_thermal_radiation_hourly,surface_pressure,...,surface_solar_radiation_downwards_hourly,surface_thermal_radiation_downwards_hourly,temperature_2m,total_evaporation_hourly,total_precipitation_hourly,u_component_of_wind_10m,v_component_of_wind_10m,volumetric_soil_water_layer_1,volumetric_soil_water_layer_2,.geo
system:index,,,,,,,,,,,,,,,,,,,,,
1999-12-31 17:00:00-07:00,2000-01-01-12,247.508530,97.571267,268.353303,268.616933,269.701919,-10262.989539,0.0,-26073.666515,101534.494640,...,0.0,781081.312563,250.773328,-0.000004,0.000110,11.263649,-5.141460,0.309007,0.301111,"{""type"":""MultiPoint"",""coordinates"":[]}"
1999-12-31 18:00:00-07:00,2000-01-01-01,247.374110,97.571267,268.357321,268.619957,269.701831,-9371.216206,0.0,-36064.010512,101546.446034,...,0.0,769369.946659,250.629647,-0.000003,0.000075,11.034984,-4.923102,0.309007,0.301111,"{""type"":""MultiPoint"",""coordinates"":[]}"
1999-12-31 19:00:00-07:00,2000-01-01-02,247.243773,97.571267,268.360946,268.623907,269.704700,-6597.208519,0.0,-61280.699030,101552.445762,...,0.0,741706.286427,250.473603,-0.000002,0.000051,11.010141,-4.892986,0.309007,0.301111,"{""type"":""MultiPoint"",""coordinates"":[]}"
1999-12-31 20:00:00-07:00,2000-01-01-03,247.228348,97.571267,268.364140,268.625852,269.705437,-5887.024561,0.0,-60714.139589,101544.532323,...,0.0,739261.117745,250.425374,-0.000002,0.000066,11.051917,-4.788566,0.309007,0.301111,"{""type"":""MultiPoint"",""coordinates"":[]}"
1999-12-31 21:00:00-07:00,2000-01-01-04,247.304836,97.571267,268.367071,268.629509,269.706259,-6051.488402,0.0,-51872.680388,101536.312677,...,0.0,746827.367444,250.490867,-0.000002,0.000126,11.075207,-4.863351,0.309007,0.301103,"{""type"":""MultiPoint"",""coordinates"":[]}"


# Satelite & Stream

In [97]:
Gaus_interp_s = 2
Gaus_interp_p = 8
Gaus_interp_d = 32

NDVI = pd.read_csv(G_Drive+'earthengine/NDVI_NBAR_FI.csv',
                              parse_dates=['date'],na_values=-9999)
NDVI = NDVI.set_index(pd.DatetimeIndex(NDVI['date']))
NDVI=NDVI.drop(['date','.geo','system:index','ndvi_FP'],axis=1)
NDVI.index = NDVI.index.tz_localize(pytz.utc).tz_convert(Mt)


GPP = pd.read_csv(G_Drive+'earthengine/GPP_Multi_Scale.csv',
                  parse_dates=['date'],na_values=-9999,index_col=['date']).dropna()
GPP.index = GPP.index.tz_localize(pytz.utc).tz_convert(Mt)

Snow = pd.read_csv(G_Drive+'earthengine/NDSI_One_Scale.csv',
                   parse_dates=['date'],na_values=['-9999'])
Snow = Snow.set_index(pd.DatetimeIndex(Snow['date']))
Snow=Snow.drop(['date','.geo','system:index'],axis=1)
Snow.index = Snow.index.tz_localize(pytz.utc).tz_convert(Mt)

Satelite = NDVI.join(Snow).join(GPP[['GPP_FI']])
# Satelite = Satelite.loc[((Satelite.index.year>=2008)&(Satelite.index.year<=2020))].copy()

print('Completeness')
Temp = Satelite.loc[((Satelite.index.month>=4)&(Satelite.index.month<=10))]
print((Temp.count()/Temp.shape[0]).round(2))


Satelite['NDSI_fill'] = Satelite['NDSI_FI'].rolling(Gaus_interp_d,center=True,min_periods=Gaus_interp_p, 
                                              win_type='gaussian').mean(std=Gaus_interp_s)

Satelite['NDSI_interp']=Satelite['NDSI_FI'].fillna(Satelite['NDSI_fill'])

Satelite['NDVI_fill'] = Satelite['ndvi_FI'].rolling(Gaus_interp_d,center=True,min_periods=Gaus_interp_p, 
                                              win_type='gaussian').mean(std=Gaus_interp_s)

Satelite['NDVI_interp']=Satelite['ndvi_FI'].fillna(Satelite['NDVI_fill'])

Satelite['GPP_interp'] = Satelite['GPP_FI'].rolling(8*4,center=True,min_periods=4, 
                                              win_type='gaussian').mean(std=Gaus_interp_s)/8

Gauge = pd.read_csv(Output_Path+'Gauge_Data.csv',
                   parse_dates=['Date'],na_values=['-9999'])
Gauge = Gauge.set_index(pd.DatetimeIndex(Gauge['Date']))
Gauge=Gauge.drop('Date',axis=1)
Gauge.loc[Gauge['EAST CHANNEL']>15,'EAST CHANNEL']=np.nan
Gauge.loc[((Gauge.index.year==2015)&(Gauge.index.dayofyear>=116)&
           (Gauge.index.dayofyear<=170)),'BIG LAKE']=np.nan
Gauge.index = Gauge.index.tz_localize(pytz.utc).tz_convert(Mt)
# Gauge = Gauge.loc[Gauge.index.year>=2008]


Data = Data_M2.join(Satelite).join(Gauge[['KULUARPAK','BIG LAKE']])

Data.head()


Completeness
ndvi_FI    0.96
NDSI_FI    0.54
GPP_FI     0.08
dtype: float64


,DOY,file_records,used_records,Tau,qc_Tau,rand_err_Tau,H,qc_H,rand_err_H,LE,...,ndvi_FI,NDSI_FI,GPP_FI,NDSI_fill,NDSI_interp,NDVI_fill,NDVI_interp,GPP_interp,KULUARPAK,BIG LAKE
2008-01-01 00:00:00-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 01:00:00-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 02:00:00-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 03:00:00-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 04:00:00-07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Fill Params

In [98]:
Data['DOY']=Data.index.dayofyear
Data['NDSI_interp'] = Data['NDSI_interp'].interpolate()
Data['NDVI_interp'] = Data['NDVI_interp'].interpolate()
